In [1]:
import numpy as np 
import pandas as pd
import findspark
import datetime
import pyspark.pandas as ps
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql import functions as F 
import warnings
warnings.filterwarnings('ignore')

findspark.init()
spark = SparkSession \
    .builder \
    .appName("df_pre") \
    .config('spark.sql.session.timeZone', 'Asia/Shanghai') \
    .master("local[*]") \
    .getOrCreate()

In [70]:
from pyspark.sql.functions import concat_ws

In [3]:
df = pd.read_csv("D:/AllData/create.csv")
spark_df = spark.createDataFrame(df)

In [82]:
spark_df.show()

+--------------+---------+------+-------+------------+--------------+-----------+-----------+---------------+-------+----+
|    PRODUCT_ID| WAFER_ID|RUN_ID|TOOL_ID| RECIPE_NAME|parametrc_name|        max|       mean|75percent_point| range1| std|
+--------------+---------+------+-------+------------+--------------+-----------+-----------+---------------+-------+----+
|AGGNBK01N.0B02|NAY514-09|    85|      1| FD/SCD/SFAQ|      NO+FG+KJ|0.016000001|0.008957386|        1.58E-6|1.12E-6|9970|
|AGGNBK01N.0B02|NAY514-09|    31|      2| FD/SCD/SFAQ|         FG+HG|      0.015|0.008611594|        1.62E-6|1.15E-6|9970|
|AGGNBK01N.0B02|NAY514-09|    85|      3| FD/SCD/SFAQ|         SS+E4|0.665000021|0.557283145|        6.61E-6|1.37E-6|9965|
|AGGNBK01N.0B02|NAY514-09|    31|      4| FD/SCD/SFAQ|         FFFFF|0.619000018| 0.52728081|        5.37E-6|1.04E-6|9965|
|AGGNBK01N.0B02|NAY514-09|    85|      1| FD/SCD/SFAQ|        SS+E4W|0.196999997|0.157893127|        9.07E-7|8.04E-7|9965|
|AGGNBK01N.0B02|

In [83]:
all_df1 = spark_df[spark_df['WAFER_ID'].isin(['NAY514-09', 'NAY514-06'])]

In [84]:
all_df1 = all_df1.withColumn('PRODUCT_ID', F.regexp_replace(col('PRODUCT_ID'), '\.', '/'))   \
                 .withColumn('WAFER_ID', F.regexp_replace(col('WAFER_ID'), '\-', '/'))  \
                 .withColumn('parametrc_name', F.regexp_replace(col('parametrc_name'), '\+', '/')) 

In [86]:
all_df1 = all_df1.withColumn("identity_column", concat_ws("#", all_df1['PRODUCT_ID'], all_df1['WAFER_ID'], 
                                                          all_df1['RUN_ID'], all_df1['parametrc_name'], all_df1['RECIPE_NAME']))

In [87]:
all_df1.show()

+--------------+---------+------+-------+------------+--------------+-----------+-----------+---------------+-------+----+--------------------+
|    PRODUCT_ID| WAFER_ID|RUN_ID|TOOL_ID| RECIPE_NAME|parametrc_name|        max|       mean|75percent_point| range1| std|     identity_column|
+--------------+---------+------+-------+------------+--------------+-----------+-----------+---------------+-------+----+--------------------+
|AGGNBK01N/0B02|NAY514/09|    85|      1| FD/SCD/SFAQ|      NO/FG/KJ|0.016000001|0.008957386|        1.58E-6|1.12E-6|9970|AGGNBK01N/0B02#NA...|
|AGGNBK01N/0B02|NAY514/09|    31|      2| FD/SCD/SFAQ|         FG/HG|      0.015|0.008611594|        1.62E-6|1.15E-6|9970|AGGNBK01N/0B02#NA...|
|AGGNBK01N/0B02|NAY514/09|    85|      3| FD/SCD/SFAQ|         SS/E4|0.665000021|0.557283145|        6.61E-6|1.37E-6|9965|AGGNBK01N/0B02#NA...|
|AGGNBK01N/0B02|NAY514/09|    31|      4| FD/SCD/SFAQ|         FFFFF|0.619000018| 0.52728081|        5.37E-6|1.04E-6|9965|AGGNBK01N/0B02

In [88]:
all_df1.dtypes

[('PRODUCT_ID', 'string'),
 ('WAFER_ID', 'string'),
 ('RUN_ID', 'bigint'),
 ('TOOL_ID', 'bigint'),
 ('RECIPE_NAME', 'string'),
 ('parametrc_name', 'string'),
 ('max', 'double'),
 ('mean', 'double'),
 ('75percent_point', 'double'),
 ('range1', 'double'),
 ('std', 'bigint'),
 ('identity_column', 'string')]

In [89]:
all_df1_spark = all_df1.pandas_api()
all_df1_spark

,PRODUCT_ID,WAFER_ID,RUN_ID,TOOL_ID,RECIPE_NAME,parametrc_name,max,mean,75percent_point,range1,std,identity_column
0,AGGNBK01N/0B02,NAY514/09,85,1,FD/SCD/SFAQ,NO/FG/KJ,0.016,0.008957,1.580000e-06,1.120000e-06,9970,AGGNBK01N/0B02#NAY514/09#85#NO/FG/KJ#FD/SCD/SFAQ
1,AGGNBK01N/0B02,NAY514/09,31,2,FD/SCD/SFAQ,FG/HG,0.015,0.008612,1.620000e-06,1.150000e-06,9970,AGGNBK01N/0B02#NAY514/09#31#FG/HG#FD/SCD/SFAQ
2,AGGNBK01N/0B02,NAY514/09,85,3,FD/SCD/SFAQ,SS/E4,0.665,0.557283,6.610000e-06,1.370000e-06,9965,AGGNBK01N/0B02#NAY514/09#85#SS/E4#FD/SCD/SFAQ
3,AGGNBK01N/0B02,NAY514/09,31,4,FD/SCD/SFAQ,FFFFF,0.619,0.527281,5.370000e-06,1.040000e-06,9965,AGGNBK01N/0B02#NAY514/09#31#FFFFF#FD/SCD/SFAQ
4,AGGNBK01N/0B02,NAY514/09,85,1,FD/SCD/SFAQ,SS/E4W,0.197,0.157893,9.070000e-07,8.040000e-07,9965,AGGNBK01N/0B02#NAY514/09#85#SS/E4W#FD/SCD/SFAQ
5,AGGNBK01N/0B02,NAY514/09,85,2,FD/SCD/SFAQ,SASA,0.191,0.161808,3.200000e-06,3.520000e-07,9965,AGGNBK01N/0B02#NAY514/09#85#SASA#FD/SCD/SFAQ
6,AGGNBK01N/0B02,NAY514/09,85,3,FD/SCD/SFAQ,SCD/PO,0.197,0.116228,2.020000e-07,1.830000e-07,9970,AGGNBK01N/0B02#NAY514/09#85#SCD/PO#FD/SCD/SFAQ
7,AGGNBK01N/0B02,NAY514/09,85,4,FD/SCD/SFAQ,SF/HGH,0.477,0.407514,1.370000e-06,3.530000e-07,9965,AGGNBK01N/0B02#NAY514/09#85#SF/HGH#FD/SCD/SFAQ
8,AGGNBK01N/0B02,NAY514/09,85,1,FD/SCD/SFAQ,DSSV/FDH/DXFS,0.207,0.163109,8.960000e-07,1.630000e-07,9965,AGGNBK01N/0B02#NAY514/09#85#DSSV/FDH/DXFS#FD/S...
9,AGGNBK01N/0B02,NAY514/09,85,2,FD/SCD/SFAQ,ZFDD/G,0.464,0.403416,2.920000e-07,8.690000e-08,9965,AGGNBK01N/0B02#NAY514/09#85#ZFDD/G#FD/SCD/SFAQ


In [52]:
all_df1_spark = all_df1_spark.astype({'PRODUCT_ID': 'string', 'WAFER_ID': 'string', 'RUN_ID': 'string',
                                     'TOOL_ID': 'string', 'RECIPE_NAME': 'string', 'parametrc_name': 'string'})

all_df1_spark['identity_column'] = ["#".join(  [ all_df1_spark['PRODUCT_ID'][i],   
                                                 all_df1_spark['RUN_ID'][i], 
                                                 all_df1_spark['RECIPE_NAME'][i],
                                                 all_df1_spark['TOOL_ID'][i], 
                                                 all_df1_spark['WAFER_ID'][i],
                                                 all_df1_spark['parametrc_name'][i]]) for i in range(len(all_df1_spark))]

all_df1_spark = all_df1_spark.astype({'identity_column': 'string'})
# all_df1_spark['identity_column'] = ["#".join([all_df1_spark['PRODUCT_ID'][i] , all_df1_spark['RUN_ID'][i] , 
#                                         all_df1_spark['TOOL_ID'][i] , all_df1_spark['RECIPE_NAME'][i] , 
#                                         all_df1_spark['parametrc_name'][i]]) for i in range(len(all_df1_spark))]

In [90]:
all_df1_spark.dtypes

PRODUCT_ID          object
WAFER_ID            object
RUN_ID               int64
TOOL_ID              int64
RECIPE_NAME         object
parametrc_name      object
max                float64
mean               float64
75percent_point    float64
range1             float64
std                  int64
identity_column     object
dtype: object

In [91]:
all_df1_spark.columns = all_df1_spark.columns.str.replace("_", "/")
all_df1_spark

,PRODUCT/ID,WAFER/ID,RUN/ID,TOOL/ID,RECIPE/NAME,parametrc/name,max,mean,75percent/point,range1,std,identity/column
0,AGGNBK01N/0B02,NAY514/09,85,1,FD/SCD/SFAQ,NO/FG/KJ,0.016,0.008957,1.580000e-06,1.120000e-06,9970,AGGNBK01N/0B02#NAY514/09#85#NO/FG/KJ#FD/SCD/SFAQ
1,AGGNBK01N/0B02,NAY514/09,31,2,FD/SCD/SFAQ,FG/HG,0.015,0.008612,1.620000e-06,1.150000e-06,9970,AGGNBK01N/0B02#NAY514/09#31#FG/HG#FD/SCD/SFAQ
2,AGGNBK01N/0B02,NAY514/09,85,3,FD/SCD/SFAQ,SS/E4,0.665,0.557283,6.610000e-06,1.370000e-06,9965,AGGNBK01N/0B02#NAY514/09#85#SS/E4#FD/SCD/SFAQ
3,AGGNBK01N/0B02,NAY514/09,31,4,FD/SCD/SFAQ,FFFFF,0.619,0.527281,5.370000e-06,1.040000e-06,9965,AGGNBK01N/0B02#NAY514/09#31#FFFFF#FD/SCD/SFAQ
4,AGGNBK01N/0B02,NAY514/09,85,1,FD/SCD/SFAQ,SS/E4W,0.197,0.157893,9.070000e-07,8.040000e-07,9965,AGGNBK01N/0B02#NAY514/09#85#SS/E4W#FD/SCD/SFAQ
5,AGGNBK01N/0B02,NAY514/09,85,2,FD/SCD/SFAQ,SASA,0.191,0.161808,3.200000e-06,3.520000e-07,9965,AGGNBK01N/0B02#NAY514/09#85#SASA#FD/SCD/SFAQ
6,AGGNBK01N/0B02,NAY514/09,85,3,FD/SCD/SFAQ,SCD/PO,0.197,0.116228,2.020000e-07,1.830000e-07,9970,AGGNBK01N/0B02#NAY514/09#85#SCD/PO#FD/SCD/SFAQ
7,AGGNBK01N/0B02,NAY514/09,85,4,FD/SCD/SFAQ,SF/HGH,0.477,0.407514,1.370000e-06,3.530000e-07,9965,AGGNBK01N/0B02#NAY514/09#85#SF/HGH#FD/SCD/SFAQ
8,AGGNBK01N/0B02,NAY514/09,85,1,FD/SCD/SFAQ,DSSV/FDH/DXFS,0.207,0.163109,8.960000e-07,1.630000e-07,9965,AGGNBK01N/0B02#NAY514/09#85#DSSV/FDH/DXFS#FD/S...
9,AGGNBK01N/0B02,NAY514/09,85,2,FD/SCD/SFAQ,ZFDD/G,0.464,0.403416,2.920000e-07,8.690000e-08,9965,AGGNBK01N/0B02#NAY514/09#85#ZFDD/G#FD/SCD/SFAQ


In [ ]:
all_df1_spark_pivot = all_df1_spark.pivot_table(index=["WAFER/ID"], columns="identity/column", 
                          values = ["max", "mean", "75percent/point", "range1"])
all_df1_spark_pivot.columns = all_df1_spark_pivot.columns.map("#".join)

In [95]:
all_df1_spark_pivot = all_df1_spark_pivot.reset_index(drop=False)
all_df1_spark_pivot

,WAFER/ID,75percent/point#AGGNBK01N/0B02#NAY514/06#31#FFFFF#FD/SCD/SFAQ,75percent/point#AGGNBK01N/0B02#NAY514/06#31#FG/HG#FD/SCD/SFAQ,75percent/point#AGGNBK01N/0B02#NAY514/06#65#SAVSD#FD/SCD/SFAQ,75percent/point#AGGNBK01N/0B02#NAY514/06#65#ZS/DSDSG#FD/SCD/SFAQW,75percent/point#AGGNBK01N/0B02#NAY514/06#65#ZSD/FG#FD/SCD/SFAQ,75percent/point#AGGNBK01N/0B02#NAY514/06#85#DSSV/FDH/DXFS#FD/SCD/SFAQ,75percent/point#AGGNBK01N/0B02#NAY514/06#85#NO/FG/KJ#FD/SCD/SFAQ,75percent/point#AGGNBK01N/0B02#NAY514/06#85#SASA#FD/SCD/SFAQ,75percent/point#AGGNBK01N/0B02#NAY514/06#85#SCD/PO#FD/SCD/SFAQ,75percent/point#AGGNBK01N/0B02#NAY514/06#85#SF/HGH#FD/SCD/SFAQ,75percent/point#AGGNBK01N/0B02#NAY514/06#85#SS/E4#FD/SCD/SFAQ,75percent/point#AGGNBK01N/0B02#NAY514/06#85#SS/E4W#FD/SCD/SFAQ,75percent/point#AGGNBK01N/0B02#NAY514/06#85#SS/HFG/DSD#FD/SCD/SFAQ,75percent/point#AGGNBK01N/0B02#NAY514/06#85#ZFDD/G#FD/SCD/SFAQ,75percent/point#AGGNBK01N/0B02#NAY514/09#31#FFFFF#FD/SCD/SFAQ,75percent/point#AGGNBK01N/0B02#NAY514/09#31#FG/HG#FD/SCD/SFAQ,75percent/point#AGGNBK01N/0B02#NAY514/09#54#DSSV/FDH/DXFS#FD/SCD/SFAQ,75percent/point#AGGNBK01N/0B02#NAY514/09#65#SAVSD#FD/SCD/SFAQ,75percent/point#AGGNBK01N/0B02#NAY514/09#65#ZS/DSDSG#FD/SCD/SFAQW,75percent/point#AGGNBK01N/0B02#NAY514/09#65#ZSD/FG#FD/SCD/SFAQ,75percent/point#AGGNBK01N/0B02#NAY514/09#85#DSSV/FDH/DXFS#FD/SCD/SFAQ,75percent/point#AGGNBK01N/0B02#NAY514/09#85#NO/FG/KJ#FD/SCD/SFAQ,75percent/point#AGGNBK01N/0B02#NAY514/09#85#SASA#FD/SCD/SFAQ,75percent/point#AGGNBK01N/0B02#NAY514/09#85#SASA/FGADD#FD/SCD/SFAQ,75percent/point#AGGNBK01N/0B02#NAY514/09#85#SCD/PO#FD/SCD/SFAQ,75percent/point#AGGNBK01N/0B02#NAY514/09#85#SF/HGH#FD/SCD/SFAQ,75percent/point#AGGNBK01N/0B02#NAY514/09#85#SS/E4#FD/SCD/SFAQ,75percent/point#AGGNBK01N/0B02#NAY514/09#85#SS/E4W#FD/SCD/SFAQ,75percent/point#AGGNBK01N/0B02#NAY514/09#85#SS/HFG#FD/SCD/SFAQ,75percent/point#AGGNBK01N/0B02#NAY514/09#85#ZFDD/G#FD/SCD/SFAQ,max#AGGNBK01N/0B02#NAY514/06#31#FFFFF#FD/SCD/SFAQ,max#AGGNBK01N/0B02#NAY514/06#31#FG/HG#FD/SCD/SFAQ,max#AGGNBK01N/0B02#NAY514/06#65#SAVSD#FD/SCD/SFAQ,max#AGGNBK01N/0B02#NAY514/06#65#ZS/DSDSG#FD/SCD/SFAQW,max#AGGNBK01N/0B02#NAY514/06#65#ZSD/FG#FD/SCD/SFAQ,max#AGGNBK01N/0B02#NAY514/06#85#DSSV/FDH/DXFS#FD/SCD/SFAQ,max#AGGNBK01N/0B02#NAY514/06#85#NO/FG/KJ#FD/SCD/SFAQ,max#AGGNBK01N/0B02#NAY514/06#85#SASA#FD/SCD/SFAQ,max#AGGNBK01N/0B02#NAY514/06#85#SCD/PO#FD/SCD/SFAQ,max#AGGNBK01N/0B02#NAY514/06#85#SF/HGH#FD/SCD/SFAQ,max#AGGNBK01N/0B02#NAY514/06#85#SS/E4#FD/SCD/SFAQ,max#AGGNBK01N/0B02#NAY514/06#85#SS/E4W#FD/SCD/SFAQ,max#AGGNBK01N/0B02#NAY514/06#85#SS/HFG/DSD#FD/SCD/SFAQ,max#AGGNBK01N/0B02#NAY514/06#85#ZFDD/G#FD/SCD/SFAQ,max#AGGNBK01N/0B02#NAY514/09#31#FFFFF#FD/SCD/SFAQ,max#AGGNBK01N/0B02#NAY514/09#31#FG/HG#FD/SCD/SFAQ,max#AGGNBK01N/0B02#NAY514/09#54#DSSV/FDH/DXFS#FD/SCD/SFAQ,max#AGGNBK01N/0B02#NAY514/09#65#SAVSD#FD/SCD/SFAQ,max#AGGNBK01N/0B02#NAY514/09#65#ZS/DSDSG#FD/SCD/SFAQW,max#AGGNBK01N/0B02#NAY514/09#65#ZSD/FG#FD/SCD/SFAQ,max#AGGNBK01N/0B02#NAY514/09#85#DSSV/FDH/DXFS#FD/SCD/SFAQ,max#AGGNBK01N/0B02#NAY514/09#85#NO/FG/KJ#FD/SCD/SFAQ,max#AGGNBK01N/0B02#NAY514/09#85#SASA#FD/SCD/SFAQ,max#AGGNBK01N/0B02#NAY514/09#85#SASA/FGADD#FD/SCD/SFAQ,max#AGGNBK01N/0B02#NAY514/09#85#SCD/PO#FD/SCD/SFAQ,max#AGGNBK01N/0B02#NAY514/09#85#SF/HGH#FD/SCD/SFAQ,max#AGGNBK01N/0B02#NAY514/09#85#SS/E4#FD/SCD/SFAQ,max#AGGNBK01N/0B02#NAY514/09#85#SS/E4W#FD/SCD/SFAQ,max#AGGNBK01N/0B02#NAY514/09#85#SS/HFG#FD/SCD/SFAQ,max#AGGNBK01N/0B02#NAY514/09#85#ZFDD/G#FD/SCD/SFAQ,mean#AGGNBK01N/0B02#NAY514/06#31#FFFFF#FD/SCD/SFAQ,mean#AGGNBK01N/0B02#NAY514/06#31#FG/HG#FD/SCD/SFAQ,mean#AGGNBK01N/0B02#NAY514/06#65#SAVSD#FD/SCD/SFAQ,mean#AGGNBK01N/0B02#NAY514/06#65#ZS/DSDSG#FD/SCD/SFAQW,mean#AGGNBK01N/0B02#NAY514/06#65#ZSD/FG#FD/SCD/SFAQ,mean#AGGNBK01N/0B02#NAY514/06#85#DSSV/FDH/DXFS#FD/SCD/SFAQ,mean#AGGNBK01N/0B02#NAY514/06#85#NO/FG/KJ#FD/SCD/SFAQ,mean#AGGNBK01N/0B02#NAY514/06#85#SASA#FD/SCD/SFAQ,mean#AGGNBK01N/0B02#NAY514/06#85#SCD/PO#FD/SCD/SFAQ,mean#AGGNBK01N/0B02#NAY514/06#85#SF/HGH#F